###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es la Policía Nacional de Colombia

In [ ]:
import pandas as pd

###Archivo d4fr-sbn2.csv

Contiene información del delito de hurto en Colombia a través de las modalidades de abigeato, bancos, y piratería desde el 01 de enero del año 2010 al 30 de abril del año 2024.


In [ ]:
df = pd.read_csv("https://www.datos.gov.co/resource/d4fr-sbn2.csv?$limit=100000")
df.head()

,departamento,municipio,codigo_dane,armas_medios,fecha_hecho,genero,grupo_etario,tipo_de_hurto,cantidad
0,HUILA,CAMPOALEGRE,41132000,NO REPORTA,1/01/2010,FEMENINO,ADULTOS,HURTO ABIGEATO,1
1,META,SAN JUAN DE ARAMA,50683000,NO REPORTA,1/01/2010,MASCULINO,ADULTOS,HURTO ABIGEATO,1
2,BOYACÁ,BELÉN,15087000,NO REPORTA,2/01/2010,FEMENINO,ADULTOS,HURTO ABIGEATO,1
3,CASANARE,PORE,85263000,NO REPORTA,2/01/2010,MASCULINO,ADULTOS,HURTO ABIGEATO,1
4,CUNDINAMARCA,MADRID,25430000,ARMA DE FUEGO,3/01/2010,MASCULINO,ADULTOS,HURTO PIRATERÍA TERRESTRE,1


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40824 entries, 0 to 40823
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   departamento   40824 non-null  object
 1   municipio      40824 non-null  object
 2   codigo_dane    40824 non-null  int64 
 3   armas_medios   40824 non-null  object
 4   fecha_hecho    40824 non-null  object
 5   genero         40809 non-null  object
 6   grupo_etario   40579 non-null  object
 7   tipo_de_hurto  40824 non-null  object
 8   cantidad       40824 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 2.8+ MB


### Eliminación de columnas irrelevantes para el proyecto

In [ ]:
relevant_cols = ['codigo_dane', 'departamento', 'municipio', 'fecha_hecho', 'genero', 'grupo_etario', 'cantidad', 'tipo_de_hurto']
theft_by_modality = df[relevant_cols]
theft_by_modality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40824 entries, 0 to 40823
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   codigo_dane    40824 non-null  int64 
 1   departamento   40824 non-null  object
 2   municipio      40824 non-null  object
 3   fecha_hecho    40824 non-null  object
 4   genero         40809 non-null  object
 5   grupo_etario   40579 non-null  object
 6   cantidad       40824 non-null  int64 
 7   tipo_de_hurto  40824 non-null  object
dtypes: int64(2), object(6)
memory usage: 2.5+ MB


### Convertir columna fecha_hecho a tipo date

In [ ]:
# Hacer una copia explícita del DataFrame
theft_by_modality_ = theft_by_modality.copy()

# Convertir la columna 'fecha_hecho' a tipo datetime usando el formato correcto
theft_by_modality_['fecha_hecho'] = pd.to_datetime(theft_by_modality_['fecha_hecho'], format='%d/%m/%Y', errors='coerce', dayfirst=True)

# Extraer solo el año
theft_by_modality_.loc[:, 'fecha_hecho'] = theft_by_modality_['fecha_hecho'].dt.year

### Verificar valores nulos

In [ ]:
theft_by_modality_.isnull().sum()

,0
codigo_dane,0
departamento,0
municipio,0
fecha_hecho,0
genero,15
grupo_etario,245
cantidad,0
tipo_de_hurto,0


Nota: Los valores nulos de genero y grupo_etario se solucionan más adelante

### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [ ]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['genero', 'grupo_etario', 'tipo_de_hurto', 'departamento', 'municipio']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(theft_by_modality_[column].unique())
    print()


Categorías en la columna 'genero':
['FEMENINO' 'MASCULINO' 'NO REPORTA' 'NO APLICA' '-' nan 'NO REPORTADO']

Categorías en la columna 'grupo_etario':
['ADULTOS' 'NO REPORTA' 'ADOLESCENTES' 'MENORES' 'NO APLICA' nan
 'NO REPORTADO']

Categorías en la columna 'tipo_de_hurto':
['HURTO ABIGEATO' 'HURTO PIRATERÍA TERRESTRE'
 'HURTO ENTIDADES FINANCIERAS']

Categorías en la columna 'departamento':
['HUILA' 'META' 'BOYACÁ' 'CASANARE' 'CUNDINAMARCA' 'SANTANDER' 'CALDAS'
 'CAUCA' 'VALLE' 'ANTIOQUIA' 'RISARALDA' 'CAQUETÁ' 'SUCRE' 'ATLÁNTICO'
 'GUAJIRA' 'MAGDALENA' 'NORTE DE SANTANDER' 'ARAUCA' 'BOLÍVAR' 'CESAR'
 'AMAZONAS' 'TOLIMA' 'VICHADA' 'NARIÑO' 'CÓRDOBA' 'PUTUMAYO' 'QUINDÍO'
 'CHOCÓ' 'VAUPÉS' 'GUAINÍA' 'GUAVIARE' 'SAN ANDRÉS']

Categorías en la columna 'municipio':
['CAMPOALEGRE' 'SAN JUAN DE ARAMA' 'BELÉN' ... 'Bucaramanga (CT)'
 'La Victoria' 'Ciénaga']



- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [ ]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [ ]:
# Aplicar la función a todas las columnas categóricas
theft_by_modality_[categorical_col] = theft_by_modality_[categorical_col].astype(str)
for col in categorical_col:
    theft_by_modality_[col] = theft_by_modality_[col].apply(remove_accents_and_special_chars)

- Mejorar consistencia de las columnas 'genero' y 'grupo_etario'

In [ ]:
theft_by_modality_['genero'] = theft_by_modality_['genero'].replace({
    'NO REPORTA': 'NO REPORTADO',
    'NO APLICA': 'NO REPORTADO',
    '-': 'NO REPORTADO',
    'NAN': 'NO REPORTADO',
    'NO REPORTADO': 'NO REPORTADO'
})

In [ ]:
theft_by_modality_['grupo_etario'] = theft_by_modality_['grupo_etario'].replace({
    'NO REPORTA': 'NO REPORTADO',
    'NO APLICA': 'NO REPORTADO',
    'NAN': 'NO REPORTADO',
    'NO REPORTADO': 'NO REPORTADO'
})

- Codificación de algunas variables categóricas

### Ajustar columna 'codigo_dane' para que coincida con el campo 'dept_mpio_code' de la tabla municipalities de la base de datos, que guarda toda la informacion de georeferenciacion de los municipios

 -  Cargar los datos con códigos reales de los municipios

Como producto de una consulta a la base de datos del proyecto que se esta construyendo (Tablas departments y municipalities) se creo el archivo csv que se carga en la siguiente celda, y que incluye los nombres de los departamentos y municipios con sus respectivos codigos, generados por el DANE (Estos codigos son los reales)

In [ ]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "codigo_dane" en el df theft_by_modality_

In [ ]:
# Asegurarnos de que todos los valores en 'codigo_dane' sean strings
theft_by_modality_['codigo_dane'] = theft_by_modality_['codigo_dane'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = theft_by_modality_['codigo_dane'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

False

In [ ]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [8 7]


In [ ]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,no_digitos_codigo_dane,count,percentage
0,8,36294,0.889036
1,7,4530,0.110964


In [ ]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = theft_by_modality_[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['codigo_dane']])
    print()

Muestra de registros con longitud 8:
  codigo_dane
0    41132000
1    50683000
2    15087000
3    85263000
4    25430000

Muestra de registros con longitud 7:
   codigo_dane
11     5172000
14     5088000
15     5001000
19     5088000
20     5129000



### Nota:

De lo anterior se notan claras inconsistencias en la columna 'codigo_dane' del df, el 88,9% tiene 8 dígitos (al parecer se le adicionaron 3 ceeros al final del código que realmente es de 5 dígitos), el 11,1% tiene solamente 7 dígitos (al parecer el cero a la izquierda de los códigos se suprimió).

  - Verificar si los codigo_dane de 7 dígitos corresponden a departamentos que se identifican con 1 dígito para validar la teoria de que al generar el dataset se les suprimió el cero a la izquierda

In [ ]:
# Filtrar las filas donde 'codigo_dane' tiene 7 dígitos
filtrado = theft_by_modality_[theft_by_modality_['codigo_dane'].str.len() == 7]

# Obtener las categorías únicas de la columna 'departamento'
categorias_departamento = filtrado['departamento'].unique()

# Imprimir las categorías
print(categorias_departamento)

['ANTIOQUIA' 'ATLANTICO']


Efectivamente ANTIOQUIA Y ATLANTICO son los unicos departamentos que tienen codigo Dane de un dígito, 5 y 8 respectivamente.

  - Adicionar un cero a los codigo_dane de 7 dígitos

In [ ]:
# Función que agrega un '0' a la izquierda si la longitud del string es 7
def add_zero_if_length_7(codigo):
    if len(codigo) == 7:
        return '0' + codigo
    return codigo

# Aplicar la función a la columna 'codigo'
theft_by_modality_['codigo_dane'] = theft_by_modality_['codigo_dane'].apply(add_zero_if_length_7)

In [ ]:
# Verificar cuantos codigo_dane de 7 dígitos quedaron
len(theft_by_modality_[theft_by_modality_['codigo_dane'].str.len() == 7])

0

  - Quitar los ultimos 3 ceros de los codigo_dane que tienen 8 dígitos

In [ ]:
# Función que corta los últimos 3 caracteres si la longitud del string es 8
def trim_last_3_if_length_8(codigo):
    if len(codigo) == 8:
        return codigo[:5]  # Dejar solo los primeros 5 caracteres
    return codigo

# Aplicar la función a la columna 'codigo'
theft_by_modality_['codigo_dane'] = theft_by_modality_['codigo_dane'].apply(trim_last_3_if_length_8)

In [ ]:
# Verificar cuantos codigo_dane de 8 dígitos quedaron
len(theft_by_modality_[theft_by_modality_['codigo_dane'].str.len() == 8])

0

- Comparar que los nombres de los departamentos en cada dataframe esten escritos correctamente

In [ ]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)
    # print(f"{label2} que no están en {label1}:")
    # print(only_in_list2)
    # print()


  - Comparar nombres de departamentos en los dataframes

In [ ]:
# Comparar listas de departamento
compare_lists(theft_by_modality_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en theft_by_modality_", "Departamentos dept_mpios_codes")

Departamentos en theft_by_modality_ que no están en Departamentos dept_mpios_codes:
{'VALLE', 'SAN ANDRES', 'GUAJIRA'}


  - Solucionar discrepancias en nombres de departamento

In [ ]:
# Diccionario de mapeo basado en los resultados de la comparación departamentos
depto_mapping = {'SAN ANDRES': 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA',
                 'VALLE': 'VALLE DEL CAUCA',
                 'GUAJIRA':'LA GUAJIRA'}

# Reemplazar los nombres incorrectos  el dataframe theft_by_modality_
theft_by_modality_['departamento'] = theft_by_modality_['departamento'].replace(depto_mapping)

 - Confirmar que discrepancia se solucionó

In [ ]:
# Comparar listas de departamento
compare_lists(theft_by_modality_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en theft_by_modality_", "Departamentos dept_mpios_codes")

Departamentos en theft_by_modality_ que no están en Departamentos dept_mpios_codes:
set()


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [ ]:
# Comparar listas de códigos
compare_lists(theft_by_modality_['codigo_dane'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en theft_by_modality_", "Códigos de municipios dept_mpios_codes")

Departamentos en theft_by_modality_ que no están en Departamentos dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [ ]:
# Eliminar columnas innecesarias
columns_to_drop = ['departamento', 'municipio']
final_theft_by_modality = theft_by_modality_.drop(columns=columns_to_drop)

In [ ]:
# Adicionar columna para trazabilidad de la fuente
final_theft_by_modality['source_id'] = 10

In [ ]:
final_theft_by_modality.columns

Index(['codigo_dane', 'fecha_hecho', 'genero', 'grupo_etario', 'cantidad',
       'tipo_de_hurto', 'source_id'],
      dtype='object')

In [ ]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'fecha_hecho': 'year_of_incident',
    'genero': 'sex',
    'grupo_etario': 'age_group',
    'cantidad': 'amount',
    'tipo_de_hurto': 'modality_of_theft',
    'codigo_dane': 'dane_code',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_theft_by_modality.rename(columns=translation_map, inplace=True)

In [ ]:
#Estructura final del dataset a integrar a la base de datos
final_theft_by_modality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40824 entries, 0 to 40823
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   dane_code          40824 non-null  object
 1   year_of_incident   40824 non-null  int32 
 2   sex                40824 non-null  object
 3   age_group          40824 non-null  object
 4   amount             40824 non-null  int64 
 5   modality_of_theft  40824 non-null  object
 6   source_id          40824 non-null  int64 
dtypes: int32(1), int64(2), object(4)
memory usage: 2.0+ MB


## Salvar en archivo csv en el drive

In [ ]:
# Guardar en archivos CSV en el drive
final_theft_by_modality.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/theft_by_modality.csv', index=False)